## Probabilistic PCA
Reference: chapter 12, Bishop book

In [106]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#from sklearn.model_selection import train_test_split
import os, pickle
import numpy as np
from numpy import log, sum, exp, prod
from numpy.linalg import det
from numpy.random import beta, binomial, dirichlet, uniform, gamma, seed, multinomial, gumbel, rand, multivariate_normal, normal
from scipy.stats import wishart #, norm, randint, bernoulli, beta, multinomial, gamma, dirichlet, uniform
from scipy.special import digamma
from imp import reload
from copy import deepcopy
#import seaborn as sns
import pandas as pd


In [116]:
N = 50000
D = 50
K = 10    # latent dimension M in book
MCsim = 50
burnin = 10

### Generate data: 

In [117]:
# DGP:

sigma2_true = 1.5
z = normal(loc=0, scale=1, size = (K,1)); z.shape
mu_true = uniform(size = (D,1)); mu.shape
W_true = uniform(0,5, size = (D,K)); W_true.shape
mean_true = (W_true @ z + mu_true).squeeze()

X = multivariate_normal(mean = mean_true, cov = sigma2_true * np.eye(D), size = N); X.shape
#X = normal(loc=0, scale=1, size = (N,D)); X.shape

(50000, 50)

In [134]:
#-------------
# Initialize:
#-------------

W = np.empty((D,K,MCsim)) 
sigma2 = np.empty((1,MCsim))           

i = 0
W[:,:,i] = normal(loc=0, scale=1, size = (D,K))
#W[:,:,i] = uniform(0,5, size = (D,K))
sigma2[0,i] = gamma(1)

x_mean = np.mean(X, axis=0)       # estimator of mu
X_mean = np.tile(x_mean, (N,1))

for i in range(1,MCsim):

    if i % 10 == 0: print("Iter: ",i)

    #--------
    # E-Step
    #--------
    M = (W[:,:,i-1].T) @ W[:,:,i-1] + (sigma2[0,i-1] * np.eye(K))
    M1 = np.linalg.inv(M) 

    Ez = M1 @ (W[:,:,i-1].T) @ (X - X_mean).T       # 12.54
    
    z_moment2 = np.empty((N,K,K)) ; x_cent = (X - X_mean).T ; cross_pr = np.empty((N,D,K))
    for n in range(N) : 
        
        z_moment2[n,:,:] = sigma2[0,i-1] * M1 + np.dot(Ez[:,n].reshape(K,1),Ez[:,n].reshape(1,K))     
        cross_pr[n,:,:] = np.dot(x_cent[:,n].reshape(D,1) , Ez[:,n].reshape(1,K))

    Ezz = z_moment2.sum(axis=0)       # 12.55
    x_Ez = cross_pr.sum(axis=0)

    #----------
    # M-step
    #----------
    #x_Ez = (X - X_mean).T @ Ez.T       # first term in 12.56

    W[:,:,i] = x_Ez @ np.linalg.inv(Ezz)     # 12.56

    # Do summing in 12.57 term-wise:
    first_term = np.sum((X - X_mean)**2)

    # 2nd term...
    second_term = -2*Ez.T.sum(axis=0) @ W[:,:,i].T @ (X - X_mean).sum(axis=0)

    # third term
    third_term = np.trace(z_moment2[i,:,:] @ W[:,:,i].T @ W[:,:,i])

    sigma2[0,i] = (first_term + second_term + third_term)/(N*D)

print('Completed!')    

# Prepare output:
sigma2 = sigma2.squeeze()
#mu = x_mean.squeeze()

Iter:  10
Iter:  20
Iter:  30
Iter:  40
Iter:  50
Completed!


In [135]:
np.mean(sigma2[burnin:])

1.4975661518863768

In [139]:
#W_true

In [141]:
W[:,:,MCsim-1]

array([[ 0.04943893,  0.03655074, -0.02238848, -0.05603587,  0.03972415,
        -0.00766235, -0.03380217,  0.00166104,  0.04651677,  0.01405712],
       [-0.02088776, -0.08117202, -0.00650318, -0.06533222,  0.06519309,
        -0.00334528,  0.00517139, -0.06863511,  0.01876773, -0.01682571],
       [-0.02100803, -0.01317328, -0.03793218,  0.01330351, -0.00474125,
         0.02533818, -0.01220109,  0.04241122, -0.02422458,  0.02764356],
       [ 0.03146032,  0.05940884,  0.01886936, -0.00023369, -0.01268728,
        -0.04285314,  0.01733184,  0.0274818 ,  0.00795539,  0.01482673],
       [ 0.00042336,  0.08240156, -0.02623319, -0.0147607 ,  0.06350572,
        -0.0049175 , -0.00456574,  0.03751048, -0.05096046,  0.01279873],
       [ 0.04758619,  0.01256191,  0.02610315, -0.00933726,  0.05267879,
         0.02027898, -0.04617896, -0.02360277,  0.04581674, -0.07694539],
       [ 0.05101548, -0.03081912,  0.01327131, -0.05113637,  0.05913026,
         0.03081436,  0.04559699,  0.04960852

In [140]:
#W.mean(axis=2)